In [1]:
import numpy as np
import cv2
import os
import subprocess
import glob
import tensorflow as tf
from util.util import generate_mask_rect, generate_mask_stroke
from net.network import GMCNNModel
import argparse
import os
import time

/root/anaconda3/envs/myconda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/root/anaconda3/envs/myconda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/root/anaconda3/envs/myconda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/root/anaconda3/envs/myconda/lib/python3.7/site-packages/tensorf

In [2]:
os.environ['CUDA_VISIBLE_DEVICES']=str(np.argmax([int(x.split()[2]) for x in subprocess.Popen(
        "nvidia-smi -q -d Memory | grep -A4 GPU | grep Free", shell=True, stdout=subprocess.PIPE).stdout.readlines()]
        ))

class TestOptions:
    def __init__(self):
        self.parser = argparse.ArgumentParser()
        self.initialized = False

    def initialize(self):
        self.parser.add_argument('--dataset', type=str, default='paris_streetview',required=False,
                                 help='The dataset of the experiment.')
        self.parser.add_argument('--data_file', type=str, default='./imgs/celebahq_512x512',required=False, help='the file storing testing file paths')
        self.parser.add_argument('--test_dir', type=str, default='./test_results',required=False, help='models are saved here')
        self.parser.add_argument('--load_model_dir', type=str, default="/mnt/inpainting_gmcnn-master/tensorflow/checkpoints/celebahq_512x512_freeform/",required=False, help='pretrained models are given here')
        self.parser.add_argument('--seed', type=int, default=1,required=False, help='random seed')
        self.parser.add_argument('--gpu_ids', type=str,required=False, default='0')

        self.parser.add_argument('--model', type=str,required=False, default='gmcnn')
        self.parser.add_argument('--random_mask', type=int, default=0,required=False,
                                 help='using random mask')

        self.parser.add_argument('--img_shapes', type=str, default='512,512,3',required=False,
                                 help='given shape parameters: h,w,c or h,w')
        self.parser.add_argument('--mask_shapes', type=str, default='32,32',required=False,
                                 help='given mask parameters: h,w')
        self.parser.add_argument('--mask_type', type=str, default='rect',required=False)
        self.parser.add_argument('--test_num', type=int, default=-1,required=False)
        self.parser.add_argument('--mode', type=str, default='save',required=False)
        self.parser.add_argument('--phase', type=str, default='test',required=False)

        # for generator
        self.parser.add_argument('--g_cnum', type=int, default=32,required=False,
                                 help='# of generator filters in first conv layer')
        self.parser.add_argument('--d_cnum', type=int, default=32,required=False,
                                 help='# of discriminator filters in first conv layer')

    def parse(self):
        if not self.initialized:
            self.initialize()
        self.opt = self.parser.parse_args(args=[])

        if self.opt.data_file != '':
            self.opt.dataset_path = self.opt.data_file

        if os.path.exists(self.opt.test_dir) is False:
            os.mkdir(self.opt.test_dir)

        assert self.opt.random_mask in [0, 1]
        self.opt.random_mask = True if self.opt.random_mask == 1 else False

        assert self.opt.mask_type in ['rect', 'stroke']

        str_img_shapes = self.opt.img_shapes.split(',')
        self.opt.img_shapes = [int(x) for x in str_img_shapes]

        str_mask_shapes = self.opt.mask_shapes.split(',')
        self.opt.mask_shapes = [int(x) for x in str_mask_shapes]

        # model name and date
        self.opt.date_str = 'test_'+time.strftime('%Y%m%d-%H%M%S')
        self.opt.model_folder = self.opt.date_str + '_' + self.opt.dataset + '_' + self.opt.model
        self.opt.model_folder += '_s' + str(self.opt.img_shapes[0]) + 'x' + str(self.opt.img_shapes[1])
        self.opt.model_folder += '_gc' + str(self.opt.g_cnum)
        self.opt.model_folder += '_randmask-' + self.opt.mask_type if self.opt.random_mask else ''
        if self.opt.random_mask:
            self.opt.model_folder += '_seed-' + str(self.opt.seed)
        self.opt.saving_path = os.path.join(self.opt.test_dir, self.opt.model_folder)

        if os.path.exists(self.opt.saving_path) is False and self.opt.mode == 'save':
            os.mkdir(self.opt.saving_path)

        args = vars(self.opt)

        print('------------ Options -------------')
        for k, v in sorted(args.items()):
            print('%s: %s' % (str(k), str(v)))
        print('-------------- End ----------------')

        return self.opt


config = TestOptions().parse()

------------ Options -------------
d_cnum: 32
data_file: ./imgs/celebahq_512x512
dataset: paris_streetview
dataset_path: ./imgs/celebahq_512x512
date_str: test_20200115-062357
g_cnum: 32
gpu_ids: 0
img_shapes: [512, 512, 3]
load_model_dir: /mnt/inpainting_gmcnn-master/tensorflow/checkpoints/celebahq_512x512_freeform/
mask_shapes: [32, 32]
mask_type: rect
mode: save
model: gmcnn
model_folder: test_20200115-062357_paris_streetview_gmcnn_s512x512_gc32
phase: test
random_mask: False
saving_path: ./test_results/test_20200115-062357_paris_streetview_gmcnn_s512x512_gc32
seed: 1
test_dir: ./test_results
test_num: -1
-------------- End ----------------


In [25]:
config.dataset_path="/mnt/inpainting_gmcnn-master/pytorch/data/test/"
config.mask_type='stroke'
pathfile = glob.glob(os.path.join(config.dataset_path, '*.jpg'))
test_num = len(pathfile)
for i in range(test_num):
    image = cv2.imread(pathfile[i])
    h, w = image.shape[:2]

    if h >= config.img_shapes[0] and w >= config.img_shapes[1]:
        h_start = (h-config.img_shapes[0]) // 2
        w_start = (w-config.img_shapes[1]) // 2
        image = image[h_start: h_start+config.img_shapes[0], w_start: w_start+config.img_shapes[1], :]
    else:
        t = min(h, w)
        image = image[(h-t)//2:(h-t)//2+t, (w-t)//2:(w-t)//2+t, :]
        image = cv2.resize(image, (config.img_shapes[1], config.img_shapes[0]))
    cv2.imwrite(os.path.join(config.dataset_path, '{:03d}.png'.format(i)),image)
pathfile = glob.glob(os.path.join(config.dataset_path, '*.png'))

In [4]:
model = GMCNNModel()

reuse = False
sess_config = tf.ConfigProto()
sess_config.gpu_options.allow_growth = False
sess=tf.Session(config=sess_config)



In [5]:
    input_image_tf = tf.placeholder(dtype=tf.float32, shape=[1, config.img_shapes[0], config.img_shapes[1], 3])
    input_mask_tf = tf.placeholder(dtype=tf.float32, shape=[1, config.img_shapes[0], config.img_shapes[1], 1])

    output = model.evaluate(input_image_tf, input_mask_tf, config=config, reuse=reuse)
    output = (output + 1) * 127.5
    output = tf.minimum(tf.maximum(output[:, :, :, ::-1], 0), 255)
    output = tf.cast(output, tf.uint8)

    # load pretrained model
    vars_list = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
    assign_ops = list(map(lambda x: tf.assign(x, tf.contrib.framework.load_variable(config.load_model_dir, x.name)),
                          vars_list))
    sess.run(assign_ops)
    print('Model loaded.')
    total_time = 0

    if config.random_mask:
        np.random.seed(config.seed)


Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Model loaded.


In [6]:
import dlib

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("/mnt/inpainting_gmcnn-master/tensorflow/shape_predictor_68_face_landmarks.dat")

In [7]:
def changelefteye(mask,pos):
    for i in range(0,21):
        for j in range(5,30):
            mask[pos[1]-i,pos[0]-j,0]=1
            mask[pos[1]+i,pos[0]-j,0]=1
    return mask

def changerighteye(mask,pos):
    for i in range(0,21):
        for j in range(5,30):
            mask[pos[1]-i,pos[0]+j,0]=1
            mask[pos[1]+i,pos[0]+j,0]=1
    return mask

In [8]:
import sys,os,traceback
import cv2
import dlib
import numpy as np

class NoFace(Exception):
    '''
    没脸
    '''
    pass

class Organ():
    def __init__(self,im_bgr,im_hsv,temp_bgr,temp_hsv,landmark,name,ksize=None):
        '''
        五官部位类
        arguments:
            im_bgr:uint8 array, inference of BGR image
            im_hsv:uint8 array, inference of HSV image
            temp_bgr/hsv:global temp image
            landmark:array(x,2), landmarks
            name:string
        '''
        self.im_bgr,self.im_hsv,self.landmark,self.name=im_bgr,im_hsv,landmark,name
        self.get_rect()
        self.shape=(int(self.bottom-self.top),int(self.right-self.left))
        self.size=self.shape[0]*self.shape[1]*3
        self.move=int(np.sqrt(self.size/3)/20)
        self.ksize=self.get_ksize()
        self.patch_bgr,self.patch_hsv=self.get_patch(self.im_bgr),self.get_patch(self.im_hsv)
        self.set_temp(temp_bgr,temp_hsv)
        self.patch_mask=self.get_mask_re()
        pass
    
    def set_temp(self,temp_bgr,temp_hsv):
        self.im_bgr_temp,self.im_hsv_temp=temp_bgr,temp_hsv
        self.patch_bgr_temp,self.patch_hsv_temp=self.get_patch(self.im_bgr_temp),self.get_patch(self.im_hsv_temp)

    def confirm(self):
        '''
        确认操作
        '''
        self.im_bgr[:],self.im_hsv[:]=self.im_bgr_temp[:],self.im_hsv_temp[:]
    
    def update_temp(self):
        '''
        更新临时图片
        '''
        self.im_bgr_temp[:],self.im_hsv_temp[:]=self.im_bgr[:],self.im_hsv[:]
        
    def get_ksize(self,rate=15):
        size=max([int(np.sqrt(self.size/3)/rate),1])
        size=(size if size%2==1 else size+1)
        return (size,size)
        
    def get_rect(self):
        '''
        获得定位方框
        '''
        ys,xs=self.landmark[:,1],self.landmark[:,0]
        self.top,self.bottom,self.left,self.right=np.min(ys),np.max(ys),np.min(xs),np.max(xs)

    def get_patch(self,im):
        '''
        截取局部切片
        '''
        shape=im.shape
        return im[np.max([self.top-self.move,0]):np.min([self.bottom+self.move,shape[0]]),np.max([self.left-self.move,0]):np.min([self.right+self.move,shape[1]])]

    def _draw_convex_hull(self,im, points, color):
        '''
        勾画多凸边形
        '''
        points = cv2.convexHull(points)
        cv2.fillConvexPoly(im, points, color=color)
        
    def get_mask_re(self,ksize=None):
        '''
        获得局部相对坐标遮罩
        '''
        if ksize==None:
            ksize=self.ksize
        landmark_re=self.landmark.copy()
        landmark_re[:,1]-=np.max([self.top-self.move,0])
        landmark_re[:,0]-=np.max([self.left-self.move,0])
        mask = np.zeros(self.patch_bgr.shape[:2], dtype=np.float64)
    
        self._draw_convex_hull(mask,
                         landmark_re,
                         color=1)
    
        mask = np.array([mask, mask, mask]).transpose((1, 2, 0))
    
        mask = (cv2.GaussianBlur(mask, ksize, 0) > 0) * 1.0
        return cv2.GaussianBlur(mask, ksize, 0)[:]
        
    def get_mask_abs(self,ksize=None):
        '''
        获得全局绝对坐标遮罩
        '''
        if ksize==None:
            ksize=self.ksize
        mask = np.zeros(self.im_bgr.shape, dtype=np.float64)
        patch=self.get_patch(mask)
        patch[:]=self.patch_mask[:]
        return mask
        
    def whitening(self,rate=0.15,confirm=True):
        '''
        提亮美白
        arguments:
            rate:float,-1~1,new_V=min(255,V*(1+rate))
            confirm:wether confirm this option
        '''
        if confirm:
            self.confirm()
            self.patch_hsv[:,:,-1]=np.minimum(self.patch_hsv[:,:,-1]+self.patch_hsv[:,:,-1]*self.patch_mask[:,:,-1]*rate,255).astype('uint8')
            self.im_bgr[:]=cv2.cvtColor(self.im_hsv, cv2.COLOR_HSV2BGR)[:]
            self.update_temp()
        else:
            self.patch_hsv_temp[:]=cv2.cvtColor(self.patch_bgr_temp, cv2.COLOR_BGR2HSV)[:]
            self.patch_hsv_temp[:,:,-1]=np.minimum(self.patch_hsv_temp[:,:,-1]+self.patch_hsv_temp[:,:,-1]*self.patch_mask[:,:,-1]*rate,255).astype('uint8')
            self.patch_bgr_temp[:]=cv2.cvtColor(self.patch_hsv_temp, cv2.COLOR_HSV2BGR)[:]
            
    def brightening(self,rate=0.3,confirm=True):
        '''
        提升鲜艳度
        arguments:
            rate:float,-1~1,new_S=min(255,S*(1+rate))
            confirm:wether confirm this option
        '''
        patch_mask=self.get_mask_re((1,1))
        if confirm:
            self.confirm()
            patch_new=self.patch_hsv[:,:,1]*patch_mask[:,:,1]*rate
            patch_new=cv2.GaussianBlur(patch_new,(3,3),0)
            self.patch_hsv[:,:,1]=np.minimum(self.patch_hsv[:,:,1]+patch_new,255).astype('uint8')
            self.im_bgr[:]=cv2.cvtColor(self.im_hsv, cv2.COLOR_HSV2BGR)[:]
            self.update_temp()
        else:
            self.patch_hsv_temp[:]=cv2.cvtColor(self.patch_bgr_temp, cv2.COLOR_BGR2HSV)[:]
            patch_new=self.patch_hsv_temp[:,:,1]*patch_mask[:,:,1]*rate
            patch_new=cv2.GaussianBlur(patch_new,(3,3),0)
            self.patch_hsv_temp[:,:,1]=np.minimum(self.patch_hsv[:,:,1]+patch_new,255).astype('uint8')
            self.patch_bgr_temp[:]=cv2.cvtColor(self.patch_hsv_temp, cv2.COLOR_HSV2BGR)[:]
        
    def smooth(self,rate=0.6,ksize=None,confirm=True):
        '''
        磨皮
        arguments:
            rate:float,0~1,im=rate*new+(1-rate)*src
            confirm:wether confirm this option
        '''
        if ksize==None:
            ksize=self.get_ksize(80)
        index=self.patch_mask>0
        if confirm:
            self.confirm()
            patch_new=cv2.GaussianBlur(cv2.bilateralFilter(self.patch_bgr,3,*ksize),ksize,0)
            self.patch_bgr[index]=np.minimum(rate*patch_new[index]+(1-rate)*self.patch_bgr[index],255).astype('uint8')
            self.im_hsv[:]=cv2.cvtColor(self.im_bgr, cv2.COLOR_BGR2HSV)[:]
            self.update_temp()
        else:
            patch_new=cv2.GaussianBlur(cv2.bilateralFilter(self.patch_bgr_temp,3,*ksize),ksize,0)
            self.patch_bgr_temp[index]=np.minimum(rate*patch_new[index]+(1-rate)*self.patch_bgr_temp[index],255).astype('uint8')
            self.patch_hsv_temp[:]=cv2.cvtColor(self.patch_bgr_temp, cv2.COLOR_BGR2HSV)[:]
        
    def sharpen(self,rate=0.3,confirm=True):
        '''
        锐化
        '''
        patch_mask=self.get_mask_re((3,3))
        kernel = np.zeros( (9,9), np.float32)
        kernel[4,4] = 2.0   #Identity, times two! 
        #Create a box filter:
        boxFilter = np.ones( (9,9), np.float32) / 81.0
        
        #Subtract the two:
        kernel = kernel - boxFilter
        index=patch_mask>0
        if confirm:
            self.confirm()
            sharp=cv2.filter2D(self.patch_bgr,-1,kernel)
            self.patch_bgr[index]=np.minimum(((1-rate)*self.patch_bgr)[index]+sharp[index]*rate,255).astype('uint8')
            self.update_temp()
        else:
            sharp=cv2.filter2D(self.patch_bgr_temp,-1,kernel)
            self.patch_bgr_temp[:]=np.minimum(self.patch_bgr_temp+self.patch_mask*sharp*rate,255).astype('uint8')
            self.patch_hsv_temp[:]=cv2.cvtColor(self.patch_bgr_temp, cv2.COLOR_BGR2HSV)[:]

class Forehead(Organ):
    def __init__(self,im_bgr,im_hsv,temp_bgr,temp_hsv,landmark,mask_organs,name,ksize=None):
        self.mask_organs=mask_organs
        super(Forehead,self).__init__(im_bgr,im_hsv,temp_bgr,temp_hsv,landmark,name,ksize)
    
    def get_mask_re(self,ksize=None):
        '''
        获得局部相对坐标遮罩
        '''
        if ksize==None:
            ksize=self.ksize
        landmark_re=self.landmark.copy()
        landmark_re[:,1]-=np.max([self.top-self.move,0])
        landmark_re[:,0]-=np.max([self.left-self.move,0])
        mask = np.zeros(self.patch_bgr.shape[:2], dtype=np.float64)
    
        self._draw_convex_hull(mask,
                         landmark_re,
                         color=1)
        
        mask = np.array([mask, mask, mask]).transpose((1, 2, 0))
    
        mask = (cv2.GaussianBlur(mask, ksize, 0) > 0) * 1.0
        patch_organs=self.get_patch(self.mask_organs)
        mask= cv2.GaussianBlur(mask, ksize, 0)[:]
        mask[patch_organs>0]=(1-patch_organs[patch_organs>0])
        return mask
        
class Face(Organ):
    '''
    脸类
    arguments:
        im_bgr:uint8 array, inference of BGR image
        im_hsv:uint8 array, inference of HSV image
        temp_bgr/hsv:global temp image
        landmarks:list, landmark groups
        index:int, index of face in the image
    '''
    def __init__(self,im_bgr,img_hsv,temp_bgr,temp_hsv,landmarks,index):
        self.index=index
        #五官名称
        self.organs_name=['jaw','mouth','nose','left eye','right eye','left brow','right brow']
        
        #五官等标记点
        self.organs_points=[list(range(0, 17)),list(range(48, 61)),list(range(27, 35)),list(range(42, 48)),list(range(36, 42)),list(range(22, 27)),list(range(17, 22))]

        #实例化脸对象和五官对象
        self.organs={name:Organ(im_bgr,img_hsv,temp_bgr,temp_hsv,landmarks[points],name) for name,points in zip(self.organs_name,self.organs_points)}

        #获得额头坐标，实例化额头
        mask_nose=self.organs['nose'].get_mask_abs()
        mask_organs=(self.organs['mouth'].get_mask_abs()+mask_nose+self.organs['left eye'].get_mask_abs()+self.organs['right eye'].get_mask_abs()+self.organs['left brow'].get_mask_abs()+self.organs['right brow'].get_mask_abs())
        forehead_landmark=self.get_forehead_landmark(im_bgr,landmarks,mask_organs,mask_nose)
        self.organs['forehead']=Forehead(im_bgr,img_hsv,temp_bgr,temp_hsv,forehead_landmark,mask_organs,'forehead')
        mask_organs+=self.organs['forehead'].get_mask_abs()

        # 人脸的完整标记点
        self.FACE_POINTS = np.concatenate([landmarks,forehead_landmark])
        super(Face,self).__init__(im_bgr,img_hsv,temp_bgr,temp_hsv,self.FACE_POINTS,'face')

        mask_face=self.get_mask_abs()-mask_organs
        self.patch_mask=self.get_patch(mask_face)
        pass
        
    
    def get_forehead_landmark(self,im_bgr,face_landmark,mask_organs,mask_nose):
        '''
        计算额头坐标
        '''
        #画椭圆
        radius=(np.linalg.norm(face_landmark[0]-face_landmark[16])/2).astype('int32')
        center_abs=tuple(((face_landmark[0]+face_landmark[16])/2).astype('int32'))
        
        angle=np.degrees(np.arctan((lambda l:l[1]/l[0])(face_landmark[16]-face_landmark[0]))).astype('int32')
        mask=np.zeros(mask_organs.shape[:2], dtype=np.float64)
        cv2.ellipse(mask,center_abs,(radius,radius),angle,180,360,1,-1)
        #剔除与五官重合部分
        mask[mask_organs[:,:,0]>0]=0
        #根据鼻子的肤色判断真正的额头面积
        index_bool=[]
        for ch in range(3):
            mean,std=np.mean(im_bgr[:,:,ch][mask_nose[:,:,ch]>0]),np.std(im_bgr[:,:,ch][mask_nose[:,:,ch]>0])
            up,down=mean+0.5*std,mean-0.5*std
            index_bool.append((im_bgr[:,:,ch]<down)|(im_bgr[:,:,ch]>up))
        index_zero=((mask>0)&index_bool[0]&index_bool[1]&index_bool[2])
        mask[index_zero]=0
        index_abs=np.array(np.where(mask>0)[::-1]).transpose()
        landmark=cv2.convexHull(index_abs).squeeze()
        return landmark
    
class Makeup():
    '''
    化妆器
    '''
    def __init__(self,predictor_path="/mnt/inpainting_gmcnn-master/tensorflow/shape_predictor_68_face_landmarks.dat"):
        self.photo_path=[]
        self.PREDICTOR_PATH = predictor_path
        self.faces={}
        
        #人脸定位、特征提取器，来自dlib
        self.detector = dlib.get_frontal_face_detector()
        self.predictor = dlib.shape_predictor(self.PREDICTOR_PATH)

    def get_faces(self,im_bgr,im_hsv,temp_bgr,temp_hsv,name,n=1):
        '''
        人脸定位和特征提取，定位到两张及以上脸或者没有人脸将抛出异常
        im:
            照片的numpy数组
        fname:
            照片名字的字符串
        返回值:
            人脸特征(x,y)坐标的矩阵
        '''
        rects = self.detector(im_bgr, 1)
        
        if len(rects) <1:
            raise NoFace('Too many faces in '+name)
        return {name:[Face(im_bgr,im_hsv,temp_bgr,temp_hsv,np.array([[p.x, p.y] for p in self.predictor(im_bgr, rect).parts()]),i) for i,rect in enumerate(rects)]}

    def read_im(self,fname,scale=1):
        '''
        读取图片
        '''
        im = cv2.imdecode(np.fromfile(fname,dtype=np.uint8),-1)
        if type(im)==type(None):
            print(fname)
            raise ValueError('Opencv error reading image "{}" , got None'.format(fname))
        return im

    def read_and_mark(self,fname):
        im_bgr=self.read_im(fname)
        im_hsv=cv2.cvtColor(im_bgr, cv2.COLOR_BGR2HSV)
        temp_bgr,temp_hsv=im_bgr.copy(),im_hsv.copy()
        return im_bgr,temp_bgr,self.get_faces(im_bgr,im_hsv,temp_bgr,temp_hsv,fname)

mu=Makeup()

In [20]:
def changeeye(mask):
    for i in range(512):
        for j in range(512):
            if mask[i][j]==1:
                for k in range(30,50):
#                     if i-k>=0 and mask[i-k][j]==0:
#                         mask[i-k][j]=2
                    if j-k>=0 and mask[i][j-k]==0:
                        mask[i][j-k]=2
                    if j+k<512 and mask[i][j+k]==0:
                        mask[i][j+k]=2
                for k in range(30,50):
                    if i+k<512 and mask[i+k][j]==0:
                        mask[i+k][j]=3
    for i in range(512):
        for j in range(512):
            if mask[i][j]==3:
                for k in range(1,10):
                    if j-k>=0 and mask[i][j-k]==0:
                        mask[i][j-k]=4
                    if j+k<512 and mask[i][j+k]==0:
                        mask[i][j+k]=4
            if mask[i][j]==2:
                for k in range(20,35):
                    if i+k<512 and mask[i+k][j]==0:
                        mask[i+k][j]=5
    for i in range(512):
        for j in range(512):
            if mask[i][j]==1:
                mask[i][j]=0
            elif mask[i][j]==2:
                mask[i][j]=1
            elif mask[i][j]==3:
                mask[i][j]=1
            elif mask[i][j]==4:
                mask[i][j]=1
            elif mask[i][j]==5:
                mask[i][j]=1
    return mask

In [21]:
def changehead(mask):
    for i in range(512):
        for j in range(512):
            if mask[i][j]==1:
                for k in range(30,50):
#                     if i-k>=0 and mask[i-k][j]==0:
#                         mask[i-k][j]=2
                    if j-k>=0 and mask[i][j-k]==0:
                        mask[i][j-k]=2
                    if j+k<512 and mask[i][j+k]==0:
                        mask[i][j+k]=2
    for i in range(512):
        for j in range(512):
            if mask[i][j]==2:
                for k in range(30,45):
                    if i-k>0 and mask[i+k][j]==0:
                        mask[i-k][j]=3
    for i in range(512):
        for j in range(512):
            if mask[i][j]==1:
                mask[i][j]=0
            elif mask[i][j]==2:
                mask[i][j]=0
            elif mask[i][j]==3:
                mask[i][j]=1
    return mask

In [22]:
def changenose(mask):
    for i in range(512):
        for j in range(512):
            if mask[i][j]==1:
                for k in range(1,10):
                    if i-k>=0 and mask[i-k][j]==0:
                        mask[i-k][j]=2
                    if i+k<512 and mask[i+k][j]==0:
                        mask[i+k][j]=2
                    if j-k>=0 and mask[i][j-k]==0:
                        mask[i][j-k]=2
                    if j+k<512 and mask[i][j+k]==0:
                        mask[i][j+k]=2
    for i in range(512):
        for j in range(512):
            if mask[i][j]==1:
                mask[i][j]=1
            elif mask[i][j]==2:
                mask[i][j]=1
    return mask

In [23]:
def changemouth(mask):
    for i in range(512):
        for j in range(512):
            if mask[i][j]==1:
                for k in range(1,10):
                    if i-k>=0 and mask[i-k][j]==0:
                        mask[i-k][j]=2
                    if i+k<512 and mask[i+k][j]==0:
                        mask[i+k][j]=2
                    if j-k>=0 and mask[i][j-k]==0:
                        mask[i][j-k]=2
                    if j+k<512 and mask[i][j+k]==0:
                        mask[i][j+k]=2
    for i in range(512):
        for j in range(512):
            if mask[i][j]==1:
                mask[i][j]=1
            elif mask[i][j]==2:
                mask[i][j]=1
    return mask

In [26]:
    for i in range(test_num):
        image = cv2.imread(pathfile[i])
        im,temp_bgr,faces=mu.read_and_mark(pathfile[i])
        print(image.shape)
        if config.mask_type == 'rect':
            mask = generate_mask_rect(config.img_shapes, config.mask_shapes, config.random_mask)
        else:
            mask = generate_mask_stroke(im_size=(config.img_shapes[0], config.img_shapes[1]),
                                        parts=8, maxBrushWidth=24, maxLength=70, maxVertex=20)
            
        mask[:,:,0]=0
        maskeye=mask.copy()
        path=pathfile[i]
        face=next(iter(faces[pathfile[i]]))
        
        #facemask=face.organs['mouth'].get_mask_abs()+face.organs['nose'].get_mask_abs()+face.organs['left eye'].get_mask_abs()+face.organs['right eye'].get_mask_abs()+face.organs['left brow'].get_mask_abs()+face.organs['right brow'].get_mask_abs()
        facemask=face.organs['left eye'].get_mask_abs()+face.organs['right eye'].get_mask_abs()
        for p in range(512):
            for q in range(512):
                if facemask[p][q].min()>0:
                    #pass
                    maskeye[p][q]=1
        maskeye=changeeye(maskeye)
        headmask=face.organs['left brow'].get_mask_abs()+face.organs['right brow'].get_mask_abs()
        maskhead=mask.copy()
        masknose=mask.copy()
        maskmouth=mask.copy()
        for p in range(512):
            for q in range(512):
                if headmask[p][q].min()>0:
                    #pass
                    maskhead[p][q]=1
        maskhead=changehead(maskhead)
        
        for p in range(512):
            for q in range(512):
                if maskhead[p][q]==1 or maskeye[p][q]==1:
                    mask[p][q]=1
        
        nosemask=face.organs['nose'].get_mask_abs()
        for p in range(512):
            for q in range(512):
                if nosemask[p][q].min()>0:
                    #pass
                    masknose[p][q]=1
        
        masknose=changenose(masknose)
        
        mouthmask=face.organs['nose'].get_mask_abs()
        for p in range(512):
            for q in range(512):
                if mouthmask[p][q].min()>0:
                    #pass
                    maskmouth[p][q]=1
        
        maskmouth=changemouth(maskmouth)
        h, w = image.shape[:2]
        for p in range(512):
            for q in range(512):
                if masknose[p][q]==1:
                    mask[p][q]=0
        if h >= config.img_shapes[0] and w >= config.img_shapes[1]:
            h_start = (h-config.img_shapes[0]) // 2
            w_start = (w-config.img_shapes[1]) // 2
            image = image[h_start: h_start+config.img_shapes[0], w_start: w_start+config.img_shapes[1], :]
        else:
            t = min(h, w)
            image = image[(h-t)//2:(h-t)//2+t, (w-t)//2:(w-t)//2+t, :]
            image = cv2.resize(image, (config.img_shapes[1], config.img_shapes[0]))

        # cv2.imwrite(os.path.join(config.saving_path, 'gt_{:03d}.png'.format(i)), image.astype(np.uint8))
        cv2.imwrite(os.path.join(config.saving_path, 'ori_{:03d}.png'.format(i)), image.astype(np.uint8))
        image = image * (1-mask) + 255 * mask
        cv2.imwrite(os.path.join(config.saving_path, 'input_{:03d}.png'.format(i)), image.astype(np.uint8))

        assert image.shape[:2] == mask.shape[:2]

        h, w = image.shape[:2]
        grid = 4
        image = image[:h // grid * grid, :w // grid * grid, :]
        mask = mask[:h // grid * grid, :w // grid * grid, :]

        image = np.expand_dims(image, 0)
        mask = np.expand_dims(mask, 0)

        result = sess.run(output, feed_dict={input_image_tf: image, input_mask_tf: mask})
        print(config.saving_path)
        cv2.imwrite(os.path.join(config.saving_path, '{:03d}.png'.format(i)), result[0][:, :, ::-1])
        print(' > {} / {}'.format(i+1, test_num))

(512, 512, 3)
./test_results/test_20200115-062357_paris_streetview_gmcnn_s512x512_gc32
 > 1 / 7
(512, 512, 3)
./test_results/test_20200115-062357_paris_streetview_gmcnn_s512x512_gc32
 > 2 / 7
(512, 512, 3)
./test_results/test_20200115-062357_paris_streetview_gmcnn_s512x512_gc32
 > 3 / 7
(512, 512, 3)
./test_results/test_20200115-062357_paris_streetview_gmcnn_s512x512_gc32
 > 4 / 7
(512, 512, 3)
./test_results/test_20200115-062357_paris_streetview_gmcnn_s512x512_gc32
 > 5 / 7
(512, 512, 3)
./test_results/test_20200115-062357_paris_streetview_gmcnn_s512x512_gc32
 > 6 / 7
(512, 512, 3)
./test_results/test_20200115-062357_paris_streetview_gmcnn_s512x512_gc32
 > 7 / 7


In [147]:
if __name__=='__main__':
    path='/mnt/inpainting_gmcnn-master/tensorflow/test_results/test_20200114-131513_paris_streetview_gmcnn_s512x512_gc32/002.png'
    mu=Makeup()
    im,temp_bgr,faces=mu.read_and_mark(path)
    imc=im.copy()
    #cv2.imshow('ori',imc)
    for face in faces[path]:
        face.whitening()
    #cv2.imshow('new',im.copy())
    
    print('Quiting')


Quiting


In [148]:
path='/mnt/inpainting_gmcnn-master/tensorflow/test_results/test_20200114-131513_paris_streetview_gmcnn_s512x512_gc32/001.png'
mu=Makeup()
im,temp_bgr,faces=mu.read_and_mark(path)
imc=im.copy()

In [76]:
face=next(iter(faces[path]))
mask=face.organs['right eye'].get_mask_abs()
for i in range(512):
    for j in range(512):
        if mask[i][j].max()>0:
            mask[i][j]=[128,0,0]

In [77]:
cv2.imwrite('face.jpg',mask)

True